In [ ]:
!pip install transformers
!pip install accelerate
!pip install sentencepiece bitsandbytes
!pip install einops timm pillow
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 130.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 100.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvj

In [ ]:
import pandas as pd
import numpy as np

import unicodedata
import string
import re

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split

import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training,PeftModel

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda:0


## Loading data and preprocessing


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/LLM Lessons/data/medquad.csv")

In [ ]:
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16412 entries, 0 to 16411
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16412 non-null  object
 1   answer      16407 non-null  object
 2   source      16412 non-null  object
 3   focus_area  16398 non-null  object
dtypes: object(4)
memory usage: 513.0+ KB


In [ ]:
#We have 5 null values on dataframe.
#Since it's impossible to predict a null object, the lines will be removed.
#We will use unsupervisioned learning to create classes using the source and focus_area column, because of that we will drop the null values on focus_area
#On answers exists a string that represents a video, we will remove this string
df = df.dropna(axis = 0, subset=['answer','focus_area'])
def remove_watch_content(text):
    # Pattern to match and remove the content inside parentheses that starts with "Watch"
    pattern = r"\(Watch(?:[^)(]+|\([^)(]*\))*\)"
    result = re.sub(pattern, '', text)
    return result
df['answer'] = df['answer'].apply(remove_watch_content)
df.reset_index(drop=True, inplace=True)
df.info()
#We still have null values on the focus_area, but we will monitorate this column

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16393 entries, 0 to 16392
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   question    16393 non-null  object
 1   answer      16393 non-null  object
 2   source      16393 non-null  object
 3   focus_area  16393 non-null  object
dtypes: object(4)
memory usage: 512.4+ KB


<ipython-input-6-a8293edbb019>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['answer'] = df['answer'].apply(remove_watch_content)


In [ ]:
## the code will use the methodology of inserting the <|symptom|> before the last ?
## on every sentence, this code will check if the last component of a list and the
## second last element will be de same, returning the index of the sentence that has duplicity
def interrogation_duplicity_check(df,column):
  for i in df[column]:
    splited_text = i.split()
    interrogation_count = splited_text.count('?')
    if interrogation_count > 1:
      print(f"Line {df.index[df[column] == i][0]} with interrogation duplicity")

interrogation_duplicity_check(df,'question')

In [ ]:
## removing the last ? on the sentence
df.loc[8082, 'question'] = ' '.join(df.iloc[8077]['question'].split()[:-1])

In [ ]:
## runing the verification again for confirmation

interrogation_duplicity_check(df,'question')

In [ ]:
def normalize_text(text):
  text = unicodedata.normalize('NFKC', text) #unicode norm
  text = text.lower()
  text = text.translate(str.maketrans('','', string.punctuation)) #remove punctuation
  text = re.sub(r'\s+', ' ', text).strip() #strip whitespaces
  return text

class TokenAdder:
    def __init__(self, token_dict=None):
        self.token_dict = token_dict or {}

    def add_tokens_to_text(self, text, custom_token=None):
        if not text:
            return text

        modified_text = text
        normalized_modified_text = normalize_text(modified_text)
        for token, strings_list in self.token_dict.items():
            # ensure strings_list is treated as a list
            if not isinstance(strings_list, list):
                strings_list = [strings_list]

            # check each string in the list
            for string_to_find in strings_list:
              normalized_string = normalize_text(string_to_find)
              if normalized_string == normalize_text("__STARTOFSENTENCE__"):
                  modified_text = f"{token}{modified_text}"
              elif normalized_string in [normalize_text(s) for s in ["__ENDOFSENTENCE__", "__ENDOFTEXT__", "__ENDOFPROMPT__"]]:
                  modified_text = f"{modified_text}{token}"
              else:
                    # regex pattern for word boundary and case-insensitive match
                    pattern = re.compile(r'\b' + re.escape(string_to_find) + r'\b', re.IGNORECASE)
                    # use a function to preserve the original case/punctuation
                    def replacer(match):
                        return f"{token}{match.group(0)}"
                    modified_text = pattern.sub(replacer, modified_text)
        return modified_text

    def process_dataframe(self, df, text_column, custom_token=None, label_column = False):
        df_copy = df.copy()
        if label_column:
          df_copy[text_column] = df_copy.apply(lambda row: f"<|{row['label']}|>{row[text_column]}", axis=1)
        df_copy[text_column] = df_copy[text_column].apply(
            lambda x: self.add_tokens_to_text(x, custom_token)
        )
        return df_copy

# Creating the Confidence tokens and grouping same classes name

- The group_similar_clsses join strings that have strong similarity, with the objective to reduce the number of classes

Confidence tokens don't exists on the dataset, but it's possible to generate them with unsupervisioned learning. We will create 4 classes:

- Reference token: present on the deseases that appears only 1 time on dataset

The 3 other classes will be classified by unsupervioned learning, the objective is to create a class that will calculate the weights based on the count of the classes on the columns "focus_area" and "source".

- Uncertain token: low level confidence answer
- Confidence token: mid level confidence answer
- High confidence token: high level confidence answer

In [ ]:
def group_similar_classes(df, column, threshold=0.8):
    classes = df[column].dropna().unique()

    # Load embedding model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Generate embeddings for each class
    embeddings = model.encode(classes)

    # Compute similarity matrix
    similarity_matrix = cosine_similarity(embeddings)

    # Identify groups of similar classes
    num_class = len(classes)
    groups = []
    visited = set()

    for i in range(num_class):
        if i in visited:
            continue

        group = [i]
        visited.add(i)

        for j in range(i + 1, num_class):
            if similarity_matrix[i, j] > threshold:
                group.append(j)
                visited.add(j)

        groups.append(group)

    # Create mapping (choosing the shortest name as the group representative)
    mapping = {}
    for group in groups:
        group_names = [classes[idx] for idx in group]

        representative_name = min(group_names, key=len)

        for name in group_names:
            mapping[name] = representative_name

    # Apply mapping and create new column
    result_df = df.copy()
    result_df[f'{column}_grouped'] = df[column].map(mapping)

    # Display grouping statistics
    print(f"Original classes: {len(classes)}")
    print(f"Classes after grouping: {len(groups)}")

    return result_df

test_df = df.copy()
test_df = group_similar_classes(test_df, 'focus_area')
df['focus_area'] = test_df['focus_area_grouped']

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Classes originais: 5125
Classes após agrupamento: 3660


In [ ]:
df_clean = df.dropna(subset=["focus_area"]).copy()
focus_counts = df_clean["focus_area"].value_counts()
df_clean["label"] = np.where(df_clean["focus_area"].map(focus_counts) == 1, "reference", None)
non_reference_df = df_clean[df_clean["label"].isnull()].copy()

class WeightedCountEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, source_weight=2.0, focus_area_weight=1.0, normalize=True):
        self.source_weight = source_weight
        self.focus_area_weight = focus_area_weight
        self.normalize = normalize

    def fit(self, X, y=None):
        self.focus_area_counts = X["focus_area"].value_counts().to_dict()
        self.source_counts = X["source"].value_counts().to_dict()

        if self.normalize:
            self.max_focus_area = max(self.focus_area_counts.values())
            self.max_source = max(self.source_counts.values())
        return self

    def transform(self, X):
        X_encoded = pd.DataFrame()

        if self.normalize:
            X_encoded["focus_area_count"] = X["focus_area"].map(self.focus_area_counts) / self.max_focus_area * self.focus_area_weight
            X_encoded["source_count"] = X["source"].map(self.source_counts) / self.max_source * self.source_weight
        else:
            X_encoded["focus_area_count"] = X["focus_area"].map(self.focus_area_counts) * self.focus_area_weight
            X_encoded["source_count"] = X["source"].map(self.source_counts) * self.source_weight

        return X_encoded

pipeline = Pipeline([
    ("encoder", WeightedCountEncoder(source_weight=1.0, focus_area_weight=4.0)),
    ("kmeans", KMeans(n_clusters=3, random_state=42))
])

cluster_labels = pipeline.fit_predict(non_reference_df[["focus_area", "source"]])
non_reference_df["cluster"] = cluster_labels

focus_area_counts = df_clean["focus_area"].value_counts()
source_counts = df_clean["source"].value_counts()

non_reference_df["score"] = (
    non_reference_df["focus_area"].map(focus_area_counts) +
    non_reference_df["source"].map(source_counts)
)

# Rank clusters by average score
cluster_scores = non_reference_df.groupby("cluster")["score"].mean().sort_values(ascending=False)
semantic_labels = {cluster: label for cluster, label in zip(cluster_scores.index, ["high_confidence", "confidence", "uncertain"])}
non_reference_df["label"] = non_reference_df["cluster"].map(semantic_labels)

final_df = pd.concat([
    non_reference_df[["question", "answer", "source", "focus_area", "label"]],
    df_clean[df_clean["label"] == "reference"][["question", "answer", "source", "focus_area", "label"]]
])

final_df.reset_index(drop=True, inplace=True)
df['label'] = final_df['label']

In [ ]:
df.head()

,question,answer,source,focus_area,label
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma,uncertain
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma,uncertain
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma,uncertain
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma,uncertain
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma,uncertain


In [ ]:
special_tokenizer_question = {
  '<think>\n':["__STARTOFSENTENCE__"],
  '<|symptom|>':["symptom", "symptoms"],
  '<|what_is|>':['what is','research','diagnose'],
  '<|quantity|>':['how many'],
  '<|causes|>':['cause', 'causes'],
  '<|risk|>': ['risk', 'risks','complications','complication'],
  '<|treatment|>': ['treatment', 'treatments','prevent'],
  '<|prevention|>': ['prevention', 'prevents', 'to do'],
  '<|endofprompt|>':["__ENDOFPROMPT__"]
}

special_add_tokenizer = TokenAdder(special_tokenizer_question)

df['question'] = special_add_tokenizer.process_dataframe(df,'question')['question']

In [ ]:
special_tokenizer_answer = {
    '<|doctor|>': ['__STARTOFSENTENCE__'],
    '<|endoftext|>': ['__ENDOFTEXT__'],
    '<|side_effects|>': ['damage','damages','lose'],
    '<|diagnosis|>': ['detected', 'test','exam','develop'],
    '<|symptoms|>': ['symptom', 'symptoms','cause','causes'],
    '<|treatment|>': ['treatment', 'treatments','normal', 'try'],
    '<|prevention|>': ['prevention', 'prevents'],
}

special_add_tokenizer = TokenAdder(special_tokenizer_answer)

df['answer'] = special_add_tokenizer.process_dataframe(df,'answer', label_column = True)['answer']

In [ ]:
special_tokens = {
    "additional_special_tokens": list(special_tokenizer_answer.keys()) + list(special_tokenizer_question.keys())
}

In [ ]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

special_tokens = {
    "additional_special_tokens": list(special_tokenizer_answer.keys()) + list(special_tokenizer_question.keys())
}
num_added_tokens = tokenizer.add_special_tokens(special_tokens)

tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    load_in_8bit=True
)

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [ ]:
model.resize_token_embeddings(len(tokenizer))

Embedding(151679, 1536)

In [ ]:
def create_dataset(df, tokenizer, max_length=512):
    inputs = []
    for _, row in df.iterrows():
        # Use your preprocessed text with DeepSeek's tokenizer
        text = f"{row['question']}{row['answer']}"
        encodings = tokenizer(
            text,
            max_length=max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        inputs.append({
            'input_ids': encodings['input_ids'][0],
            'attention_mask': encodings['attention_mask'][0],
            'labels': encodings['input_ids'][0].clone()
        })
    return inputs

In [ ]:
train_sentence, test_sentence = train_test_split(df, test_size=0.4, random_state=42)
test_sentence, val_sentence = train_test_split(test_sentence, test_size=0.5, random_state=42)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results_deepseek_medical",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    gradient_accumulation_steps=4
)


In [ ]:
train_dataset = create_dataset(train_sentence, tokenizer)
val_dataset = create_dataset(val_sentence, tokenizer)
test_dataset = create_dataset(test_sentence, tokenizer)

In [ ]:
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

In [ ]:
model = prepare_model_for_kbit_training(model)

In [ ]:
model = get_peft_model(model, lora_config)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
)


<ipython-input-27-62670ca281bb>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: augusto-cesar-rodrigues (augusto-cesar-rodrigues-undb-unidade-de-ensino-superior-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Epoch,Training Loss,Validation Loss
1,1.417000,1.399557
2,1.207900,1.211656


/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/peft/

TrainOutput(global_step=921, training_loss=1.4402645019962008, metrics={'train_runtime': 4886.8205, 'train_samples_per_second': 6.038, 'train_steps_per_second': 0.188, 'total_flos': 1.3992393091055616e+17, 'train_loss': 1.4402645019962008, 'epoch': 2.991869918699187})

In [ ]:
model.save_pretrained("./deepseek_medical_qa_peft")
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.save_pretrained("./deepseek_medical_qa_peft")

/usr/local/lib/python3.11/dist-packages/peft/utils/save_and_load.py:250: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


('./deepseek_medical_qa_peft/tokenizer_config.json',
 './deepseek_medical_qa_peft/special_tokens_map.json',
 './deepseek_medical_qa_peft/tokenizer.json')